In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import matplotlib.pyplot as plt

A_to_au_conversion = 1.8897259885789

In [2]:
def calc_adapt_vqe_N2H4(threshold, active_electrons=4, active_orbitals=4):
    symbols = ["N", "H", "H", "N", "H", "H"]
    # ground state coordinate: 
    geometry = np.array([0.0, 0.0, 0.0 , 0.0, 0.0, 1.015264, 0.978541, 0.0, -0.270591, -0.627449, 1.276052, -0.477492 , -0.897827, 1.825923, 0.332013 , 0.080714, 1.825923, -0.953842])*A_to_au_conversion

    H, qubits = qchem.molecular_hamiltonian(
        symbols,
        geometry,
        active_electrons=active_electrons,
        active_orbitals=active_orbitals,
        # method="pyscf"
    )

    active_electrons = active_electrons

    singles, doubles = qchem.excitations(active_electrons, qubits)

    print(f"Total number of excitations = {len(singles) + len(doubles)}")
    singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
    doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
    operator_pool = doubles_excitations + singles_excitations   
    hf_state = qchem.hf_state(active_electrons, qubits)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit():
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
        return qml.expval(H)
    energy_array = []
    opt = qml.optimize.AdaptiveOptimizer()
    for i in range(len(operator_pool)):
        circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool)
        energy_array.append(energy)
        # if i % 3 == 0:
        print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
        # print(qml.draw(circuit, decimals=None)())
        print()
        if gradient < threshold*10^(-threshold):
            break
    return energy_array, circuit


In [3]:
config = [[12, 7]]
threshold = 3

for i in range(len(config)):
    print("Configuration: ", threshold, config[i][0], config[i][1])
    E, circuit = calc_adapt_vqe_N2H4(threshold, active_electrons=config[i][0], active_orbitals=config[i][1])
    
    file_path = f"data/N2H4_{threshold}_{config[i][0]}_{config[i][1]}.txt"
    # Open the file in write mode
    with open(file_path, "w") as file:
        # Write each element of the array to a new line
        for element in E:
            file.write(str(element) + "\n")

Configuration:  3 12 7
Total number of excitations = 48
n = 0,  E = -109.73067811 H, Largest Gradient = 0.087

n = 1,  E = -109.73619567 H, Largest Gradient = 0.068

n = 2,  E = -109.74023838 H, Largest Gradient = 0.048

n = 3,  E = -109.74217454 H, Largest Gradient = 0.047

n = 4,  E = -109.74403294 H, Largest Gradient = 0.039

n = 5,  E = -109.74468342 H, Largest Gradient = 0.021

n = 6,  E = -109.74519842 H, Largest Gradient = 0.017

n = 7,  E = -109.74538637 H, Largest Gradient = 0.014

n = 8,  E = -109.74550605 H, Largest Gradient = 0.007

n = 9,  E = -109.74553639 H, Largest Gradient = 0.007

n = 10,  E = -109.74556420 H, Largest Gradient = 0.006

n = 11,  E = -109.74559971 H, Largest Gradient = 0.006

n = 12,  E = -109.74563414 H, Largest Gradient = 0.005

n = 13,  E = -109.74565255 H, Largest Gradient = 0.004

n = 14,  E = -109.74566431 H, Largest Gradient = 0.003

n = 15,  E = -109.74567296 H, Largest Gradient = 0.003

n = 16,  E = -109.74569582 H, Largest Gradient = 0.003

n 